In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Module 2. Векторная модель текста и классификация длинных текстов ###

**2.3.1**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b = symbols('x y w b')
y_hat = 1/(1+exp(-w*x-b))
eq = -y*log(y_hat)-(1-y)*log(1-y_hat)
print(diff(eq, w).simplify())

In [ ]:
import sympy.parsing.sympy_parser

sample_expr_str = 'x*(-y*exp(-b - w*x) - y + 1)/(exp(-b - w*x) + 1)'
sample_expr = sympy.parsing.sympy_parser.parse_expr(sample_expr_str)
sample_value = sample_expr.evalf(subs=dict(x=0.5, y=1, w=4, b=1))
print(sample_value)

**2.3.2**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b = symbols('x y w b')
y_hat = 1/(1+exp(-w*x-b))
eq = -y*log(y_hat)-(1-y)*log(1-y_hat)
print(diff(eq, b).simplify())

**2.3.3**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b, c = symbols('x y w b, c')
y_hat = 1/(1+exp(-w*x-b))
reg = c * (w**2 + b**2)
eq = -y*log(y_hat)-(1-y)*log(1-y_hat) + reg
print(diff(eq, w).simplify())

**2.3.6**

In [ ]:
import sys
import numpy as np

def parse_array(s):
    return np.array([int(s.strip()) for s in s.strip().split(' ')])

def read_array(inp):
    return parse_array(inp)

def calculate_pmi(a, b):
    #p(a,b) - вероятность того, что оба события совершились одновременно (и там и там единица).
    #p(a) , p(b) - вероятности реализации случайных событий.
    #pmi(a,b) = log(p(a,b)/(p(a)*p(b)))
    p_a_b = np.mean(a * b)
    p_a = np.mean(a)
    p_b = np.mean(b)
    return np.log(p_a_b / (p_a * p_b))  

a = read_array('1 0 0 1 1 0')
b = read_array('1 0 0 0 1 0')
pmi_value = calculate_pmi(a, b)

print('{:.6f}'.format(pmi_value))

**2.3.7**

In [ ]:
import numpy as np
s = 2
P = 10 / 10000 #заданная плотность распределения
Z = np.array([i**(-2) for i in range(1, 1001)]).sum()
rank = (1 / (Z * 10 / 10000))**(1/s)
num = int(1000 - rank) + 1
num

**2.4.3**

In [ ]:
import re
import sys

text = [
'Контактный телефон: 123123.',
'Что-нибудь надо придумать.',
'Значение числа Е=2.7182.',
'Демон123, как тебя зовут в реале?',
'-1-.15=-1.15',
'- 1 - .15 = -1.15',
'Какого ;%:?* тут происходит?'
]

# модифицируйте это регулярное выражение
parts = [
    r'[а-яё]+',
    r'-?\d*[.,]?\d+',
    r'\S'
]
#re.I -  re.IGNORECASE - Perform case-insensitive matching
#Флаг, избавляет нас от необходимости указывать заглавные буквы алфавита 
#при выделении слов и достаточно указать лишь диапазон строчных букв.
TOKENIZE_RE = re.compile(r'|'.join(parts), re.I) 


def tokenize(txt):
    return TOKENIZE_RE.findall(txt)


for line in text:
    print(' '.join(tokenize(line.strip().lower())))

**2.4.5**

In [ ]:
import re
import sys
import collections
import numpy as np



def tokenize_text(text, lower=True):
    parts = [
        r'[\w\d]+'
    ]
    TOKENIZE_RE = re.compile(r'|'.join(parts), re.I)
    if lower:
        return TOKENIZE_RE.findall(text.lower())
    else:
        return TOKENIZE_RE.findall(text)

def tokenize_corpus(texts, lower=True):
    return [tokenize_text(text, lower=lower) for text in texts]

def build_vocabulary(tokenize_texts):
    word_counts = collections.defaultdict(int)
    
    for text in tokenize_texts:
        unique_tokens = set(text)
        for token in unique_tokens:
            word_counts[token] += 1
    doc_count = len(tokenize_texts)
    
    word_counts_sorted = sorted(word_counts.items(), reverse=False, key=lambda kv:(kv[1], kv[0]))
    
    word2id = {word: i for i, (word, _) in enumerate(word_counts_sorted)}
    word2freq = np.array([word_count/doc_count for (_, word_count) in word_counts_sorted], dtype='float32')
    
    return word2id, word2freq

texts = [
"Казнить нельзя, помиловать. Нельзя наказывать.",
"Казнить, нельзя помиловать. Нельзя освободить.",
"Нельзя не помиловать.",
"Обязательно освободить."
]

tokenized_texts = tokenize_corpus(texts)
voc_word2id, voc_word2freq = build_vocabulary(tokenized_texts)

print(' '.join(voc_word2id.keys()))
print(' '.join(map(str, voc_word2id.values())))
print(' '.join(map(str, voc_word2freq)))

In [ ]:
# Федор Краснов
# 2 years ago
# Привет,
# для любопытных: с помощью sklearn тоже можно сделать, но есть особенность, которой и делюсь.

corpus = [
    'Казнить нельзя, помиловать. Нельзя наказывать.',
    'Казнить, нельзя помиловать. Нельзя освободить.',
    'Нельзя не помиловать.',
    'Обязательно освободить.']

## Получение df
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=False, use_idf=True)
vectorizer.fit_transform(corpus)
word_doc_freq = 1/np.exp(vectorizer.idf_ - 1) ## ВОТ ОСОБЕННОСТЬ ПЕРЕВОДА ИЗ IDF В DF

# Словарь тут: 
# vectorizer.get_feature_names()

**2.4.7**

In [ ]:
import scipy.sparse
import numpy as np
def vectorize_texts_ltfidf(tokenized_texts, voc_word2id, voc_word2freq):
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(voc_word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in voc_word2id.keys():
                result[text_i, voc_word2id[token]] += 1
    result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
    result = result.multiply(1/result.sum(1))
    result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
    result = result.multiply(1 / voc_word2freq)
    feats_std = result.toarray().std(0, ddof=1)
    feats_mean = result.toarray().mean(0)
    return (result.toarray() - feats_mean) / feats_std

In [ ]:
vectorize_texts = vectorize_texts_ltfidf(tokenized_texts, voc_word2id, voc_word2freq)
[print(' '.join(map(str, row))) for row in vectorize_texts];

In [ ]:
# Salavat Garifullin
from collections import Counter
import pandas as pd
import re

corpus = ['Казнить нельзя, помиловать. Нельзя наказывать.',
         'Казнить, нельзя помиловать. Нельзя освободить.',
         'Нельзя не помиловать.',
         'Обязательно освободить.']
TOKENIZE_RE = re.compile(r'[\w\d]+')

corpus = pd.Series(corpus).str.lower().apply(TOKENIZE_RE.findall)
corpus_counts = corpus.apply(Counter).apply(pd.Series).fillna(0).astype(int)
word2freq = (corpus_counts > 0).mean(0).sort_index().sort_values()

### ^ код из 5 шага

import numpy as np

TF = (corpus_counts[word2freq.index].T / corpus.apply(len)).T
lTFIDF = np.log(TF + 1) / word2freq
feature_matrix  = (lTFIDF - lTFIDF.mean()) / lTFIDF.std(0, ddof=1)

for doc in feature_matrix.round(2).values:
    print(*doc)

**2.4 Final task**

In [ ]:
ДЗ по семинару реализовано в отдельном ноутбуке: stepik-nn-and-nlp 2.4.final (bag-of-words-models).ipynb

### Module 3. Базовые нейросетевые методы для работы с текстами ###

**3.3 Final task**

In [ ]:
ДЗ по семинару реализовано в отдельном ноутбуке: stepik-nn-and-nlp 3.3 final (word-embeddings).ipynb

**3.4.1**

In [ ]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def generate_w2v_sgns_samples(text, window_size, vocab_size, ns_rate):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample

    returns list of training samples (CenterWord, CtxWord, Label)
    """
    rwindow = window_size // 2
    output = []
    for i in range(len(text)):
        pos_sample = [*text[max(i-rwindow,0):i], *text[i+1:i+rwindow+1]]
        output.extend([[text[i], pos_id, 1] for pos_id in pos_sample])
        output.extend([[text[i], neg_id, 0] for neg_id in np.random.randint(vocab_size, size=ns_rate*len(pos_sample))])
    return output

text = np.random.randint(100,size=100)
window_size = 7
vocab_size = 100
ns_rate = 3

result = generate_w2v_sgns_samples(text, window_size, vocab_size, ns_rate)

#write_array(np.array(result))

**3.4.2**

In [ ]:
def update_w2v_weights(center_embeddings, context_embeddings, center_word, context_word, label, learning_rate):
    """
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_word - int - identifier of center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    #здесь для истории, заменил свой код (которым решил задание), 
    #кодом Эдуарда Смирнова, т.к. он более читабельный (сама суть аналогична)
    w = center_embeddings[center_word]
    d = context_embeddings[context_word]

    sigma = 1/(1+np.exp(-w@d))
    grad_w = (sigma-label) * d
    grad_d = (sigma-label) * w

    w -= learning_rate * grad_w
    d -= learning_rate * grad_d

**3.4.3**

In [ ]:
import sys
import ast
import numpy as np


def read_list():
    return ast.literal_eval(sys.stdin.readline())

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample
    token2subwords - list of lists of int - i-th sublist contains list of identifiers of n-grams for token #i (list of subword units)

    returns list of training samples (CenterSubwords, CtxWord, Label)
    """
    rwindow = window_size // 2
    output = []
    for i in range(len(text)):
        pos_sample = [*text[max(i-rwindow,0):i], *text[i+1:i+rwindow+1]]
        output.extend([[[text[i], *token2subwords[text[i]]], pos_id, 1] for pos_id in pos_sample])
        output.extend([[[text[i], *token2subwords[text[i]]], neg_id, 0] for neg_id in np.random.randint(vocab_size, size=ns_rate*len(pos_sample))])
    return output


text = [1, 2, 0, 1, 4, 0, 4, 1, 5, 4, 5, 4, 5, 1]
window_size = 3
vocab_size = 6
ns_rate = 2
token2subwords = [[17], [10, 12], [20, 20], [7, 13], [], [7, 11]]

result = generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords)

print(repr(result))

**3.4.4**

In [ ]:
def update_ft_weights(center_embeddings, context_embeddings, center_subwords, context_word, label, learning_rate):
    """
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_subwords - list of ints - list of identifiers of n-grams contained in center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    #заменил для истории на подход от Дани Крутовкий, у меня было чуть длинее, за счет того,
    #что я забыл, что для выбора нескольких значений сразу можно слайс подавать
    w_mean = center_embeddings[center_subwords].mean(-2)
    d = context_embeddings[context_word]

    sigma = 1/(1+np.exp(-w_mean@d))
    grad_w = (sigma-label) * d
    grad_d = (sigma-label) * w_mean
    
    context_embeddings[context_word] -= learning_rate * grad_d
    center_embeddings[center_subwords] -= learning_rate * grad_w / len(center_subwords)

**3.4.5**

In [ ]:
import sys
import ast
import numpy as np
import scipy.sparse


def read_array():
    return ast.literal_eval(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def generate_coocurrence_matrix(texts, vocab_size):
    """
    texts - list of lists of ints - i-th sublist contains identifiers of tokens in i-th document
    vocab_size - int - size of vocabulary
    returns scipy.sparse.dok_matrix
    """
    co_matrix = scipy.sparse.dok_matrix((vocab_size, vocab_size), dtype=np.float32)
    for doc in text:
        tokens = tuple(set(doc))
        for i in tokens:
            co_matrix[i,tokens] += 1   
    co_matrix.setdiag(0)
    return co_matrix

text = [[0, 2, 2, 2, 0, 0], [1, 1, 2, 1, 1], [2, 2, 1, 1]]
vocab_size = 3

result = generate_coocurrence_matrix(text, vocab_size)

write_array(result.toarray())

In [ ]:
import sys
import ast
import numpy as np
import scipy.sparse


def read_array():
    return ast.literal_eval(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def generate_coocurrence_matrix(texts, vocab_size):
    """
    texts - list of lists of ints - i-th sublist contains identifiers of tokens in i-th document
    vocab_size - int - size of vocabulary
    returns scipy.sparse.dok_matrix
    """
    co_matrix = scipy.sparse.dok_matrix((vocab_size, vocab_size), dtype=np.float32)
    for doc in text:
        tokens = tuple(set(doc))
        mesh = np.array(np.meshgrid(tokens, tokens)).reshape(2,-1)
        co_matrix[mesh[0],mesh[1]] += 1
    co_matrix.setdiag(0)
    return co_matrix

text = [[0, 2, 2, 2, 0, 0], [1, 1, 2, 1, 1], [2, 2, 1, 1]]
vocab_size = 3

result = generate_coocurrence_matrix(text, vocab_size)

write_array(result.toarray())

**3.4.6**

In [ ]:
def weight(x, max_x, alpha):
    mask = (x <= max_x)
    out = np.ones(x.shape)
    out[mask] = (x[mask]/max_x)**alpha
    return out

def update_glove_weights(x, w, d, alpha, max_x, learning_rate):
    """
    x - square integer matrix VocabSize x VocabSize - coocurrence matrix
    w - VocabSize x EmbSize - first word vectors
    d - VocabSize x EmbSize - second word vectors
    alpha - float - power in weight smoothing function f
    max_x - int - maximum coocurrence count in weight smoothing function f
    learning_rate - positive float - size of gradient step
    """
    f = weight(x, max_x, alpha)
    
    pre = f*2*(w@d.T - np.log1p(x))
    grad_w = pre @ d
    grad_d = pre.T @ w
    
    w -= learning_rate * grad_w
    d -= learning_rate * grad_d

**3.4.7**

In [ ]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def get_nearest(embeddings, query_word_id, get_n):
    """
    embeddings - VocabSize x EmbSize - word embeddings
    query_word_id - integer - id of query word to find most similar to
    get_n - integer - number of most similar words to retrieve

    returns list of `get_n` tuples (word_id, similarity) sorted by descending order of similarity value
    """
    norm_emb = (embeddings.T / (np.linalg.norm(embeddings,axis=-1) + 1e-9)).T
    sims = -np.linalg.norm(norm_emb-norm_emb[query_word_id], axis=-1)
    return [[arg, sims[arg]] for arg in np.argsort(-sims)[:get_n]]


embeddings = np.array([[0.7299015792584768, 0.2915364327741303, 0.5307571134639943, 0.3101345732086396, 
                        0.8327085262119636, 0.39018382511314353, 0.678094726221033, 0.12372148102696612, 
                        0.5966533433209616], [0.5411155947267721, 0.046791742239819856, 0.5358832195593092, 
                        0.09894162419462038, 0.6350557173679914, 0.15126161842015717, 0.11375720216711405, 
                        0.46954553941325416, 0.8281402097264261], [0.5323869209381028, 0.2005012376766715,
                        0.5925043884236925, 0.4621530177251649, 0.3886830034303448, 0.6403738184472031, 
                        0.23320289120963578, 0.43574647265888766, 0.5305633832484254]])
query_word_id = 0
get_n = 8

result = get_nearest(embeddings, query_word_id, get_n)

write_array(np.array(result))

**3.6.2**

In [ ]:
import numpy as np
kernel = np.array((-0.5,0,0.5))
signal = np.array((1,1,2,3,3,3,2,1,1), dtype=np.float32)
" ".join(map(str,([signal[i:i+len(kernel)]@kernel for i in range(len(signal) - len(kernel)+1)])))

**3.6.4**

In [ ]:
import numpy as np
kernel = np.array([[1,1,0],
                   [0,1,1]], dtype=np.float32) #Kernel(InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)
bias = 0
out = [bias + X[i:i+kernel.shape[1],:].flatten()@kernel.T.flatten() for i in range(X.shape[0] - kernel.shape[1]+1)]
" ".join(map(str,out))

In [ ]:
import numpy as np
kernel = np.array([[1,1,0],
                   [0,1,1]], dtype=np.float32) #Kernel(InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)
bias = 0
out = [bias + np.sum(X[i:i+kernel.shape[1],:]*kernel.T) for i in range(X.shape[0] - kernel.shape[1]+1)]
" ".join(map(str,out))

**3.6.5**

In [ ]:
import numpy as np
kernel = np.array([[[1,1,0],
                   [0,1,1]],
                  [[1,0,0],
                   [0,0,1]]], dtype=np.float32) #Kernel(OutChannels,InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)
bias = 0
out = np.array([bias + np.sum(X[i:i+kernel.shape[2],:]*kernel.transpose((0,2,1)), axis=(1,2)) \
                for i in range(X.shape[0] - kernel.shape[2]+1)]).reshape(-1,kernel.shape[0])
print("\n".join(map(lambda row: str(row)[1:-1],out)))

In [ ]:
#решение от Роман Григоров
#Обновил на более оптимальное решение.
#Разик делаем матричное умножение, потом суммы диагоналей с шагом считаем
import numpy as np

kernel = np.array([[[1,1,0],
                   [0,1,1]],
                  [[1,0,0],
                   [0,0,1]]], dtype=np.float32) #Kernel(OutChannels,InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)

x_kernel_product = X @ kernel
y = np.array([np.trace(x_kernel_product, pos, 2) for pos in range(X.shape[0] - kernel.shape[2]+1)])
y

**3.7.Разбор видео**

In [ ]:
import numpy as np

kern_size = 3
sequence_len = 7
in_channel = 4

x = np.array([[1,2,3,4,5,6,7],
              [10,20,30,40,50,60,70],
              [1,2,3,4,5,6,7],
              [1,2,3,4,5,6,7]])
chunks = []
chunk_size = sequence_len - kern_size + 1
for offset in range(kern_size):
    chunks.append(x[:,offset:offset+chunk_size])
chunks

**3.8.1**

In [ ]:
def apply_convolution(data, kernel, bias):
    """
    data - InLen x InChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutLen x OutChannels
    """
    return np.array([bias + np.sum(data[i:i+kernel.shape[2],:]*kernel.transpose((0,2,1)), axis=(1,2)) \
                for i in range(data.shape[0] - kernel.shape[2]+1)]).reshape(-1,kernel.shape[0])


**3.8.2**

In [ ]:
def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """
    k_ones = np.ones(kernel.shape).transpose((0,2,1))
    return np.sum([x[i:i+k_ones.shape[1],:]*k_ones for i in range(x.shape[0] - k_ones.shape[1]+1)],axis=0).transpose((0,2,1))

In [ ]:
def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """
    return np.repeat(np.sum([x[i:i+kernel.shape[2],:] for i in range(x.shape[0] - kernel.shape[2]+1)],axis=0)\
          .transpose((1,0))[np.newaxis,:,:],kernel.shape[0],axis=0)

**3.8.3**

In [ ]:
def calculate_conv_x_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns InLen x InChannels
    """
    stkernel = np.sum(kernel, axis=0).T
    x_grad = np.zeros(x.shape)
    for i in range(x.shape[0] - kernel.shape[2]+1):
        x_grad[i:i+kernel.shape[2],:] += stkernel
    return x_grad

**3.8.4**

In [ ]:
def calculate_receptive_field(layers):
    """
    layers - list of LayerInfo

    returns int - receptive field size
    """
    return 1 + sum(map(lambda x: (x.kernel_size-1)*x.dilation, layers))